In [3]:
import os
import random
import numpy as np
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import scipy.stats
import yaml

from data import VideoIterator, ClickAnnotation, file_loader

class PATHS:
    
    config_path = "config.yaml"

    @classmethod
    def load_config(cls):
        """Load paths from YAML file"""
        with open(cls.config_path, "r") as file:
            config = yaml.safe_load(file)

        cls.video_name = config["video_name"]
        cls.video_path = config["video_path"].format(video_name=cls.video_name)
        cls.annotation_path = config["annotation_path"]
        cls.save_dirs = config["save_dirs"]

    # load clicks from path
    @classmethod
    def load_clicks(cls):
        user_folders = [f for f in os.listdir(cls.annotation_path) if os.path.isdir(os.path.join(cls.annotation_path, f))]
        all_clicks = []
        for i, user_folder in enumerate(user_folders):
            clicks = file_loader(os.path.join(cls.annotation_path, user_folder, cls.video_name))
            all_clicks.append(clicks)
        
        return all_clicks
    
    def set_video_name(cls, new_name):
        cls.video_path = cls.video_path.replace(cls.video_name, new_name)
        cls.video_name = new_name


    @classmethod
    def user_names(cls):
        return [f for f in os.listdir(cls.annotation_path) if os.path.isdir(os.path.join(cls.annotation_path, f))]
    
    @classmethod
    def video_names(cls):
        return [f for f in os.listdir(os.path.join(cls.annotation_path, cls.user_names()[0])) if os.path.isdir(os.path.join(cls.annotation_path, cls.user_names()[0], f))]

PATHS.load_config()

In [4]:
video_names = PATHS.video_names()
for video_name in video_names:
    PATHS.set_video_name(PATHS, video_name)
    video = VideoIterator(PATHS.video_path)
    annotations = ClickAnnotation(PATHS.annotation_path, PATHS.video_name ,interpolate=True, sequence_length=video.num_frames+1)
    break


number of clicks: 5723
Number of annotators 8

In [73]:
xall_ann = []
yall_ann = []
# for i in range(8):

for ann in annotations:
    xflat_ann = [user_clicks[0] for user_clicks in ann[:4]]
    yflat_ann = [user_clicks[1] for user_clicks in ann[:4]]
    xall_ann.append(xflat_ann)
    yall_ann.append(yflat_ann)
xall_flat_ann = np.array(xall_ann)
yall_flat_ann = np.array(yall_ann)

# calcule a variância das distâncias ao centro
xvar = xall_flat_ann.var(axis=1)
# xvar = xall_flat_ann.var(axis=1)
xmean = xall_flat_ann.mean(axis=1)
ymean = yall_flat_ann.mean(axis=1)

svar = ((xall_flat_ann - xmean.reshape(-1, 1))**2 + (yall_flat_ann - ymean.reshape(-1, 1))**2).sum(axis=1)/len(xall_flat_ann)
xall_flat_ann.std(axis=1)

array([39.09283822, 39.09283822, 39.09283822, ..., 55.91679086,
       44.61221806, 39.09283822])

In [70]:
(xall_flat_ann.var(axis=1) + yall_flat_ann.var(axis=1))

array([ 4369.5   ,  4369.5   ,  4369.5   , ..., 11577.9375, 10576.75  ,
        4369.5   ])